In [1]:
import pickle
a = open("Articles.pickle",'rb')
uncleanedarticles = pickle.load(a)
articles=[]
text=""
title=""
titles=[]
sources=[]
for i in uncleanedarticles:
    source,url,title,text = i[0],i[1],i[2],i[3]
    sources.append(source)
    text = text.replace('\n\n','\n')
    if text =="":
        continue
#     article = i.split('\n\n\n\n\n')
#     text = article[1].strip('\n').strip(' ')
    if text.count('\n') <10:
        continue
    if text.count('https://') >10:
        continue   
#     title = article[0].strip('\n').strip(' ')
    title = title.strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([source,url,title,text])
a.close()
# print(articles)

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}

articleslen=[]
for i in articles:
    title,text = i[2],i[3]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [31]:
# print(len(wordRepArticles))
# print(wordRepArticles[0])
print(len(articles))
print(len(uncleanedarticles))
fileObject = open('./mysite/cleanedarticles.pickle','wb')
pickle.dump(articles,fileObject)
fileObject.close()

2352
6198


In [4]:
articles[:3]

[['abcnews',
  'https://abcnews.go.com/US/reads-monday/story?id=59712958',
  'Must reads before Monday',
  'One parent\'s decision to shame his daughter on social media drew mixed responses, Florida is struggling with major toxic algae blooms, the only operating spaceport in the world is crumbling, and eight children affected by Trump\'s "zero-tolerance" border policy are still waiting to be reunited with their families.\nDon’t miss out on some of ABC News\' best reporting from this week.\n\'What are you trying to prove?\': Parenting experts weigh in on dad\'s bullying punishment\nSTOCK/Getty Images\nA viral video posted by a father that forced his daughter to walk 5 miles to school in near-freezing temperatures has prompted questions about the use of social media as a way to humiliate and discipline children. Some experts say that shame is not an "effective way to change behavior."\n\'When is this going to stop?\': Florida activist highlights devastating impact of algae blooms on wild

In [5]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(n):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,n):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [6]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][2])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][2])

Main title: Must reads before Monday
0/3: Father denied visa for slain daughter's funeral as suspect charged with rape, murder
1/3: Man accused of killing 13-year-old Hania Aguilar could face death penalty if convicted
2/3: FBI: Man arrested in death of North Carolina teenager
Main title: Worcester firefighter dies while battling huge blaze, the latest in a string of firefighter deaths in city
0/3: Chief identifies firefighter who died after becoming trapped in five-alarm Worcester fire
1/3: 5 children under age of 10 die in Ohio house fire
2/3: California marks 1 month since deadly wildfires ripped through state
Main title: Funeral for 'Baby Jane Doe' found near California highway
0/3: Parents charged in 10-month-old daughter's overdose death
1/3: Texas Daycare Worker Accused of Breaking Baby’s Bones
2/3: Boyfriend allegedly kills girlfriend and her 17-year-old daughter: Sheriff
Main title: Amazon touted as big win for NY, but math is more complex
0/3: CVS offers 'guaranteed net cost'

1/3: Defending DJ: Was the fatal shooting of a Pace University football player justified?
2/3: 'Shame on You': Ingraham Blasts New Jersey AG for Rules Limiting Police Cooperation With ICE
Main title: Childish Gambino’s ‘This is America’ video is a beautiful nightmare
0/3: 2019 Grammy nominees: Kendrick Lamar, Drake, Lady Gaga, Cardi B, Taylor Swift
1/3: H.E.R., Ella Mai, Mendes and more react to the Grammy noms
2/3: 5 Things You Didn't Know About Kat Timpf
Main title: 50 Years Later, We Still Don't Grasp the Mother of All Demos
0/3: How Doug Engelbart Pulled off the Mother of All Demos
1/3: iPhone: The Complete History—and What's Next
2/3: Canada Welcomes AI, But Not All AI Researchers
Main title: While You Were Offline: Maybe All Social Media Was a Bad Idea
0/3: Gadget Lab Podcast: What the Tumblr Porn Ban Means for Users
1/3: Tumblr is eliminating pornography and, with it, a more freewheeling era of the internet
2/3: What Michael Flynn's memo means: Today's talker
Main title: Waymo’s

Main title: Steven Mnuchin under consideration to be next Trump chief of staff
0/3: Nick Ayers out as candidate to become President Donald Trump's next chief of staff
1/3: Pence's chief of staff will not take over White House role after John Kelly leaves
2/3: President Trump scrambles for next chief of staff with John Kelly set to depart
Main title: Mueller court filing details how Manafort allegedly lied to the government
0/3: Mueller to release new details on ex-Trump aides Manafort, Cohen
1/3: Special counsel says Paul Manafort lied "on a variety of subject matters," breaching plea deal
2/3: Paul Manafort lied about contact with administration officials: Special counsel
Main title: Mueller says Michael Cohen helped special counsel's probe
0/3: Mueller to release new details on ex-Trump aides Manafort, Cohen
1/3: Trump calls for end to Mueller probe despite Russian campaign bid findings
2/3: Russia-related cases: Your guide to this week's action on Michael Cohen, Paul Manafort and Ma

Main title: Broken laws, Russia negotiations: Can Trump presidency survive latest court filings?
0/3: Mueller says Michael Cohen helped special counsel's probe
1/3: Mueller is close to answering Russia collusion question that could end Trump's presidency
2/3: Chris Christie: President Trump 'not totally clear' until special counsel Robert Mueller 'hands in the keys'
Main title: John Kelly, hired to restore order for President Donald Trump, is out as chief of staff
0/3: Amid reports of John Kelly departure, Donald Trump team signals coming staff announcement
1/3: Pence's chief of staff will not take over White House role after John Kelly leaves
2/3: Trump says chief of staff John Kelly will leave at the end of the year
Main title: Do we need the government to tell us that almond milk doesn't come from a cow?
0/3: Eat healthier without quitting sugar
1/3: How many have to be hospitalized for salmonella, E. coli before we try food irradiation?
2/3: Healthier sides and desserts for your ho

Main title: Five bombshells from the Michael Cohen memos, including information he's given Mueller
0/3: Mueller says Michael Cohen helped special counsel's probe
1/3: Key takeaways from the latest round of court documents in Mueller's investigation
2/3: Special counsel memo reveals information on Michael Cohen's Russia contacts
Main title: Robert Mueller: Cohen provided details 'core' to inquiry into Russian coordination with Trump campaign
0/3: Mueller says Michael Cohen helped special counsel's probe
1/3: 'Thank you': Donald Trump claims vindication after Mueller investigation updates
2/3: Key takeaways from the latest round of court documents in Mueller's investigation
Main title: Paul Manafort: Special counsel Robert Mueller details lies that doomed plea deal
0/3: Russia-related cases: Your guide to this week's action on Michael Cohen, Paul Manafort and Maria Butina
1/3: Mueller to release new details on ex-Trump aides Manafort, Cohen
2/3: Special counsel says Paul Manafort lied "o

...

Main title: Brexit is just six months away. And neither backers nor foes are happy.
0/3: U.K. PM May warns of 'uncharted waters' if lawmakers reject Brexit deal
1/3: Brexit will put focus on 'talents and skills,' not E.U. passports, May says
2/3: Brexit deal is worse than staying in E.U., lawmaker who helped draft it says
Main title: Architect of two-state solution doubts Trump can deliver Mideast peace
0/3: Why Palestinians risk their lives to protest near Gaza-Israel fence
1/3: Amid Trump's Mideast push, an abandoned building reminds Palestinians of past failures
2/3: As Israel moves right, young American Jews target Birthright tours in protest
Main title: Turkey will conduct 'operations' against Erdogan's enemies in U.S.
0/3: Turkey's Erdogan to build more prisons as post-coup purge persists
1/3: Mueller gives new details on Flynn's secretive work for Turkey
2/3: Wife of NASA scientist jailed in Turkey 'frustrated' by Trump's focus on pastor's case
Main title: Iceland's proposed ban

Main title: Communist-era secret police files reopen old wounds in Albania
0/3: Sergei Skripal's would-be assassins will likely escape justice
1/3: Pakistan suicide bomber on motorcycle kills 25 at market
2/3: Refugees called an Olympic venue home while looking for the real thing
Main title: Cyprus turns off taps to farmers as fresh water levels drop
0/3: The Dead Sea is dying. A $1.5 billion plan aims to resurrect it.
1/3: Water scarcity fuels tensions across the Middle East
2/3: German farmers sue government over missed climate targets
Main title: Aquarius ordeal highlights hardening views on migrants in Europe
0/3: Aquarius migrant rescue ship stops Mediterranean Sea patrols
1/3: Italy orders seizure of Aquarius rescue ship
2/3: ‘Doctors Without Borders’ Permanently Cancels Migrant Ferry Operations
Main title: Divorce on the rise in Iraq as wives cut ties to ISIS militants
0/3: Pakistan suicide bomber on motorcycle kills 25 at market
1/3: Trump admin may send captured ISIS fighters 

0/3: Are You Putting Money in Your 401k? Jim Cramer Says Now's the Time
1/3: Michael Kors: Still on the cutting edge
2/3: Why Dolce & Gabbana's China blunder could be such a disaster
Main title: Amazing travel job: Director of fly fishing, Grande Lakes Orlando
0/3: Fisherman criticized by fellow anglers, animal activists over 1,400-pound marlin catch
1/3: South Carolina fisherman hooks 17-foot great white shark: It was a 'monster'
2/3: The most remote fine dining experience in the world?
Main title: The Science of Growing a Perfect Christmas Tree
0/3: North Carolina is for (Christmas) tree lovers
1/3: 10 travel-worthy Christmas trees across the USA
2/3: How to choose the best Christmas tree
Main title: Why We All Take the Same Travel Photos
0/3: Amsterdam removes 'I Amsterdam' sculpture from city center to stem influx of selfie-snapping tourists
1/3: 5 famous sights that are illegal to photograph
2/3: The female photographers who vanished from museums
Main title: The Mueller Investigat

2/3: Mueller says Michael Cohen helped special counsel's probe
Main title: UK government delays Parliament's vote on Brexit
0/3: U.K. PM May warns of 'uncharted waters' if lawmakers reject Brexit deal
1/3: What Investors Need to Know About the Brexit Deal
2/3: Theresa May says she will delay a planned vote on Brexit despite furious Parliament
Main title: Comey says voters need to hold Trump accountable in 2020 'landslide,' opposes impeachment
0/3: Democrats 'have to win' in 2020: James Comey responds to recent Trump attacks in new interview
1/3: Pirro on 'Russian Collusion Delusion': 'Time for the Left to Fold Up Their Fantasy Tents'
2/3: James Comey: Russia investigation started with four Americans with ties to Trump, Russia
Main title: New data shows asylum claims spiking at U.S. ports of entry
0/3: Military troops to remain at border through January, says Defense Secretary James Mattis
1/3: The number of migrant families captured entering illegally increased in November
2/3: Migrant

Main title: Apple slapped with sales and import ban in China on older iPhone models
0/3: Legal Quarrel With Qualcomm Adds to Apple Shareholder Anxiety Over China
1/3: Apple Turns Higher After China Court Grants Injunction Banning iPhone Sales
2/3: China's Ban on Apple's Older iPhones Is Deeply Ironic -- and Unsettling
Main title: Dow gives up 200 points as bank shares fall and trade war concerns intensify
0/3: Dow Jones drop adds to losses, weekly gloom for stock investors
1/3: White House trade adviser says impact of tariffs on stock market a 'false narrative'
2/3: Dow reverses course and ends higher despite trade, Brexit concerns
Main title: Elon Musk says Tesla would consider buying idled GM plants, takes another swing at SEC
0/3: Tesla could buy GM's shuttered plants, Elon Musk says
1/3: Musk suggests Tesla's new chairwoman won't rein him in
2/3: Elon Musk: 'I do not respect the SEC'
Main title: The 10 biggest mergers and acquisitions of 2018
0/3: 2019 best places to work revealed 

In [7]:
print(avgArticleLen)

972.2436224489796


In [8]:
sentmatch = []
for i in range(n):
    mainarticle = wordRepArticles[i]
    rep=[]
    for j in range(matches):
        comparearticle = wordRepArticles[topmatches[i][j][1]]
        bestmatchSentence =[]
        for k in range(len(mainarticle[2])):
            val = -1
            msentence = mainarticle[2][k][1]
            mainlength = len(mainarticle[2][k][0].split(' '))
            for l in range(len(comparearticle[2])):
                csentence = comparearticle[2][l][1]
                comparelength = len(comparearticle[2][l][0].split(' '))
                commonwords = list(set(msentence.keys()) & set(csentence.keys())) 
                value = 0
                for word in commonwords:
                    mainvalue = (1+math.log(msentence[word]))*1.0/(math.log(1+mainlength/avgArticleLen))
                    compvalue = (1+math.log(csentence[word]))*1.0/(math.log(1+comparelength/avgArticleLen))
                    value += mainvalue*compvalue* math.log(1+n/idf[word])
                if value>val:
                    val = value
                    ind = l
            bestmatchSentence.append([ind,val,comparearticle[2][ind][0]])
        rep.append(bestmatchSentence)
    sentmatch.append(rep)

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 

reliability = [9.9 for i in range(len(sources))]
articleaggreeement=[]
for i in range(n):
    articleval =0
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        val=0
        comparticle=wordRepArticles[topmatches[i][j][1]]
        for k in range(len(mainarticle[2])):
            sent1 = analyzer.polarity_scores(mainarticle[2][k][0])['compound']
            sent2 = analyzer.polarity_scores(sentmatch[i][j][k][2])['compound']
            val +=sent1*sent2
        val /= len(mainarticle[2])
        articleval +=val*reliability[sources.index(articles[topmatches[i][j][1]][0])]
    articleaggreeement.append(articleval/matches)
    articles[i].append(articleval/matches)

In [18]:
# fileObject = open('cleanedarticles.pickle','wb')
# pickle.dump(articles,fileObject)
# fileObject.close()
print(articles[:2])

[['abcnews', 'https://abcnews.go.com/US/reads-monday/story?id=59712958', 'Must reads before Monday', 'One parent\'s decision to shame his daughter on social media drew mixed responses, Florida is struggling with major toxic algae blooms, the only operating spaceport in the world is crumbling, and eight children affected by Trump\'s "zero-tolerance" border policy are still waiting to be reunited with their families.\nDon’t miss out on some of ABC News\' best reporting from this week.\n\'What are you trying to prove?\': Parenting experts weigh in on dad\'s bullying punishment\nSTOCK/Getty Images\nA viral video posted by a father that forced his daughter to walk 5 miles to school in near-freezing temperatures has prompted questions about the use of social media as a way to humiliate and discipline children. Some experts say that shame is not an "effective way to change behavior."\n\'When is this going to stop?\': Florida activist highlights devastating impact of algae blooms on wildlife\n

In [32]:
sourcerelia=[]
for i in range(len(sources)):
    sourcerelia.append([sources[i],reliability[i]])
fileObject = open('./mysite/sourcereliability.pickle','wb')
pickle.dump(sourcerelia,fileObject)
fileObject.close()
# print(sourcerelia[0])

In [33]:
articlesimilarity=[]
for i in range(n):
    for j in range(len(topmatches[i])):
#         articlesimilarity.append([articles[i][2],articles[topmatches[i][j][1]][2]])
        articlesimilarity.append([i,topmatches[i][j][1],topmatches[i][j][0]])
fileObject = open('./mysite/articlessimilarity.pickle','wb')
pickle.dump(articlesimilarity,fileObject)
fileObject.close()

In [34]:
print(articlesimilarity[0])

[0, 400, 1796.743738974511]


In [29]:
# fileObject = open('./mysite/articlessimilarity.pickle','rb')
# a=pickle.load(fileObject)
# fileObject.close()

In [30]:
# a[0]

[0, 400, 1796.743738974511]